In [1]:
from deepface import DeepFace

def get_gender(img_path, shorten=True):
    result = DeepFace.analyze(img_path = img_path,
        actions = ['gender']
    )
    if not shorten:
        return result
    else:
        return result[0]['dominant_gender']

2024-04-30 13:42:46.674679: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-30 13:42:46.700002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 13:42:47.058763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from pathlib import Path 
from tqdm import tqdm
import cv2  
import os 
map_gender = {'man': 0, 'woman': 1}
all_path = [str(x) for x in Path('/home/ptthang/UARK CLASS/privacy_face_protect/CelebA_HQ_face_gender_dataset_test_1000').glob('*/*.jpg')]
saved_dir_blur = '/home/ptthang/UARK CLASS/privacy_face_protect/CelebA_HQ_face_gender_dataset_test_1000_blur'
os.makedirs(saved_dir_blur, exist_ok=True)
for path in tqdm(all_path):
    try:
        img = cv2.imread(path)
        img = cv2.resize(img, (128, 128))
        img = cv2.GaussianBlur(img, (25, 25), 0)
        os.makedirs(saved_dir_blur + '/' + path.split('/')[-2], exist_ok=True)
        cv2.imwrite(saved_dir_blur + '/' + path.split('/')[-2] + '/' + path.split('/')[-1], img)
    except:
        print('Error:', path)

all_path = [str(x) for x in Path(saved_dir_blur).glob('*/*.jpg')]
total = len(all_path)
predictions = [] 
groundtruth = [] 
correct = 0
failure = []
for path in tqdm(all_path):
    try:
        gender = get_gender(path).lower()
        predictions.append(map_gender[gender])
        groundtruth.append(map_gender[path.split('/')[-2]])
        if map_gender[gender] == map_gender[path.split('/')[-2]]:
            correct += 1
    except:
        failure.append(path)
print('Accuracy:', correct/total)

  0%|          | 0/142 [00:00<?, ?it/s]2024-04-30 13:43:59.310295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 13:43:59.313417: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
100%|██████████| 142/142 [00:23<00:00,  6.06it/s]

Accuracy: 0.6971830985915493


In [ ]:
import os 
for f in failure:
    os.remove(f)

In [ ]:
path.split('/')[-2], gender

('man', 'man')